In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

import pandas
pandas.set_option('display.max_rows', 1000)

from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler

import lsst.geom
import lsst.afw.display as afwDisplay

# TAP

## List all the tables

In [ ]:
service = get_tap_service()

In [ ]:
# results = service.search("SELECT description, table_name FROM TAP_SCHEMA.tables")
# results_tab = results.to_table()
# results_tab

## Which RA/Dec are covered by this test area? Use Object table to find out.

Using our typical central-to-DC2 coordinates as a default search region doesn't work, so just grab a bunch of bright Objects and see where they are.

In [ ]:
# results = service.search("SELECT * from TAP_SCHEMA.columns WHERE table_name = 'dp02_test_PREOPS863_00.Object'")
# print(results.fieldnames)

In [ ]:
# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp02_test_PREOPS863_00.Object'")
# results.to_table().to_pandas()

In [ ]:
# results = service.search("SELECT coord_ra, coord_dec, detect_isPrimary, r_calibFlux "\
#                          "FROM dp02_test_PREOPS863_00.Object "\
#                          "WHERE detect_isPrimary = 1 AND r_calibFlux > 1000000 ", \
#                          maxrec=100000)
# results_tab = results.to_table()

In [ ]:
# print(len(results_tab))
# results_tab

In [ ]:
# plt.plot( results_tab['coord_ra'], results_tab['coord_dec'], 'o', ms=3, alpha=0.3, mew=0 )

So we can use 57.5, -36.5. as the approximate central coordinate for searches when needed.

## DiaObject

In [ ]:
# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp02_test_PREOPS863_00.DiaObject'")
# results.to_table().to_pandas()

In [ ]:
results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_fluxToAbMag(gPSFluxMax/1e32) AS gPSMagMin, "
                         "scisql_fluxToAbMag(rPSFluxMax/1e32) AS rPSMagMin, "
                         "scisql_fluxToAbMag(iPSFluxMax/1e32) AS iPSMagMin "
                         "FROM dp02_test_PREOPS863_00.DiaObject "\
                         "WHERE scisql_fluxToAbMag(gPSFluxMax/1e32) > 14 "
                         "AND scisql_fluxToAbMag(gPSFluxMax/1e32) < 24 "
                         "AND scisql_fluxToAbMag(rPSFluxMax/1e32) > 14 "
                         "AND scisql_fluxToAbMag(rPSFluxMax/1e32) < 24 "
                         "AND scisql_fluxToAbMag(iPSFluxMax/1e32) > 14 "
                         "AND scisql_fluxToAbMag(iPSFluxMax/1e32) < 24 "
                         "AND CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 0.5)) = 1 ", maxrec=10000)
DiaObjs = results.to_table()
del results

In [ ]:
print(len(DiaObjs))

In [ ]:
# DiaObjs['rPSMagMean'] = -2.50 * np.log10(DiaObjs['rPSFluxMean']) + 31.4

In [ ]:
# DiaObjs

## DiaSource

In [ ]:
# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp02_test_PREOPS863_00.DiaSource'")
# results.to_table().to_pandas()

## Truth

In [ ]:
# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp01_dc2_catalogs.truth_match'")
# results.to_table().to_pandas()

In [ ]:
# Define the query
query = "SELECT id, ra, dec, match_objectId, truth_type, redshift, mag_r "\
        "FROM dp01_dc2_catalogs.truth_match "\
        "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 0.5)) = 1 "\
        "AND redshift < 0.4 AND redshift > 0.01 AND truth_type = 3"
print(query)

In [ ]:
t1 = time.time()
results = service.search(query, maxrec=10000)
t2 = time.time()
print('Time elapsed: ',t2-t1,' seconds')
print(len(results))

Truth = results.to_table()
del results

In [ ]:
Truth

In [ ]:
fig = plt.figure(figsize=(14,14))
plt.plot(DiaObjs['ra'], DiaObjs['decl'], 'o', ms=8, alpha=0.8, mew=0, color='dodgerblue')
plt.plot(Truth['ra'], Truth['dec'], '*', ms=5, alpha=1, color='black')
plt.show()

In [ ]:
index_truth = []
index_diaobj = []

cosdec = np.cos(np.radians(36.6))
for i in range(len(Truth)):
    tx = np.where( (np.absolute(Truth['ra'][i] - DiaObjs['ra']) < 3.0/3600.0) & \
                   (np.absolute(Truth['dec'][i] - DiaObjs['decl'])*cosdec < 3.0/3600.0)  )[0]
    if len(tx) > 0:
        index_truth.append(i)
        index_diaobj.append(tx[j])
        print('match: ', Truth['id'][i], Truth['redshift'][i], len(tx))
        for j in range(len(tx)):
            print([j],DiaObjs['diaObjectId'][tx[j]],
                  DiaObjs['gPSMagMin'][tx[j]], 
                  DiaObjs['rPSMagMin'][tx[j]], 
                  DiaObjs['iPSMagMin'][tx[j]])
    del tx

In [ ]:
all_filters = ['u','g','r','i','z','y']
all_filtclr = ['darkviolet','darkgreen','red','darkorange','brown','black']

fig, ax = plt.subplots( len(index_truth), figsize=(14,14), sharey=False, sharex=False )

for j in range(len(index_truth)):
    sel_diaObjectId = DiaObjs['diaObjectId'][index_diaobj[j]]
    results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, filterName, midPointTai, psFlux, psFluxErr "
                             "FROM dp02_test_PREOPS863_00.DiaSource "
                             "WHERE diaObjectId = "+str(sel_diaObjectId))
    results = results.to_table()
    results['psAbMag'] = -2.50 * np.log10(results['psFlux']) + 31.4
    
    for f,filt in enumerate(all_filters):
        fx = np.where( results['filterName'] == filt )[0]
        ax[j].plot( results['midPointTai'][fx], results['psAbMag'][fx], 'o', ms=5, mew=0, alpha=0.8, color=all_filtclr[f] )
    
    ax[j].set_ylim([np.max(results['psAbMag'])+0.3, np.min(results['psAbMag'])-0.3])
    
    del results

<br><br>

# Butler

In [ ]:
# repo = 's3://butler-us-central1-dp01'
repo = 's3://butler-us-central1-panda-dev/dc2/butler-external.yaml'
butler = dafButler.Butler(repo)

In [ ]:
registry = butler.registry

In [ ]:
for c in sorted(registry.queryCollections()):
    print(c)